In [ ]:
import json
with open("outputs/buckets07-22_16:37.json","r") as file:
  buckets = json.load(file)
print(len(buckets))

In [ ]:

bucket_names_to_bucket={bucket[0]:bucket for bucket in buckets}
display(bucket_names_to_bucket)

In [ ]:
# Prepare batch to come up with names for each bucket
import random
import numpy as np
import json

with open("outputs/buckets07-22_16:37.json","r") as file:
  buckets = json.load(file)

bucket_names_to_bucket={bucket[0]:bucket for bucket in buckets}
bucket_names=[bucket[0] for bucket in buckets]
print
manual_name_to_llm_reponse={}
random.seed(1423)
bucket_names_str="\n".join(random.sample(bucket_names,k=20))
BATCH_SIZE=600
bucket_names_batches = np.array_split(bucket_names, range(BATCH_SIZE, len(bucket_names), BATCH_SIZE))
action_i=0
for batch_i in range(len(buckets)//BATCH_SIZE+1):
  with open(f"./outputs/llm_name_buckets/07-22_16:37/input/batch{batch_i}.jsonl","w") as file:
    for bucket_name in bucket_names_batches[batch_i]:
      input=f"""I am analyzing a dataset full of Congressional actions. I have categorized these actions into buckets,\
      where each bucket has near-identical actions, usually differing only in the names/numbers of congresspeople,bills,votes,and committees. \
      I want to come up with names for these buckets. These names are usually just the actions with names/numbers of congresspeople, bills, votes and committees removed.
      First, here is a sample of one action from the 20 buckets chosen at random:\n
      {bucket_names_str}\n
      Here are some example names you might give to bucket actions:\n
      Action from bucket: Mr. Whitfield moved that the Committee now rise.\n
      Bucket name: Congressperson moved the Committee now rise.\n\n
      Action from bucket: The House resumed with the motion to agree in the Senate amendment to H.R. 83, with an amendment. (consideration: CR H9284-9290)\n
      Bucket name: The House resumed with the motion to agree in the Senate amendment to the bill, with an amendment.\n\n
      Action from bucket: Subcommittee Consideration and Mark-up Session Held and Forwarded to Full Committee by the Subcommittee on Capital Markets and Government Sponsored Enterprises Prior to Introduction and Referral Revised discussion draft, as amended, ordered favorably reported to the full committee by the subcommittee on Capital Markets and Government Sponsored Enterprises\n
      Bucket name: Subcommittee Consideration and Mark-up Session Held and Forwarded to Full Committee by the Subcommittee Prior to Introduction and Referral. Revised discussion draft, as amended, ordered favorably reported to the full committee by the subcommittee\n\n
      Action from bucket: Passed Senate with amendments by Yea-Nay. 93 - 7. Record Vote Number: 218.\n
      Bucket name: Passed Senate with amendments by Yea-Nay.\n\n
      
      Now, please come up with a name for the bucket with the following actions. Please only give me the name, nothing else in your response.
      If there is only one line, it means the bucket has only one action:\n
      {"\n".join(random.sample(bucket_names_to_bucket[bucket_name],k=20) if len(bucket_names_to_bucket[bucket_name])>20 else bucket_names_to_bucket[bucket_name])}"""
    
      # input_len+=len(input)
      json.dump({"custom_id":"bucket"+str(action_i),"url":"/v1/responses","method":"POST","body":{"input":input,"model":"gpt-4.1-mini"}}, file)
      file.write("\n")
      action_i+=1
    # manual_name_to_llm_reponse[bucket_name]=client.responses.create(input=input,model="gpt-4.1-mini")
    # print(bucket_name)
  # print(input_len)

In [ ]:
# Load OpenAI client
import dotenv
import os
import openai
import asyncio
import json
import time
dotenv.load_dotenv()
client = openai.Client(api_key=os.environ["OPENAI_API_KEY"])
statuses=[batch.status for batch in client.batches.list()]
while "cancelling" in statuses or "in_progress" in statuses:
  time.sleep(60)
  statuses=[batch.status for batch in client.batches.list()]
print("starting batches")
for batch_i in range(2,8):
  with open(f"./outputs/llm_name_buckets/07-22_16:37/input/batch{batch_i}.jsonl","rb") as file:
    batch_file=client.files.create(file=file,purpose="batch")
  batch=client.batches.create(input_file_id=batch_file.id,endpoint="/v1/responses",completion_window="24h")
  print(batch.id)
  while client.batches.retrieve(batch.id).status!="completed":
    if batch.status=="failed":
      break
    time.sleep(60)
  with open(f"./outputs/llm_name_buckets/07-22_16:37/output/batch{batch_i}.jsonl","w") as file:
    file.write(client.files.content(file_id=client.batches.retrieve(batch_id=batch.id).output_file_id).text)
  print(f"finished batch {batch_i}")